# User Models

> User Models

- skip_showdoc: true
- skip_exec: true

## Custom User Model

> Create a custom user model by extending AbstractUser

```python
# base/models.py
from django.contrib.auth.models import AbstractUser
from django.db import models

class CustomUser(AbstractUser):
    # Add additional fields if necessary
    phone_number = models.CharField(max_length=15, blank=True)

```

## Update settings.py to use the custom user model

```python
# myproject/settings.py
AUTH_USER_MODEL = 'base.CustomUser'

```

## User Registration and Authentication

### Serializers

> Create serializers for user registration and authentication

```python
# base/serializers.py
from rest_framework import serializers
from django.contrib.auth import get_user_model
from django.contrib.auth.password_validation import validate_password

User = get_user_model()

class UserRegistrationSerializer(serializers.ModelSerializer):
    password = serializers.CharField(write_only=True, required=True, validators=[validate_password])
    password2 = serializers.CharField(write_only=True, required=True)

    class Meta:
        model = User
        fields = ('username', 'password', 'password2', 'email', 'first_name', 'last_name')

    def validate(self, attrs):
        if attrs['password'] != attrs['password2']:
            raise serializers.ValidationError({"password": "Password fields didn't match."})
        return attrs

    def create(self, validated_data):
        user = User.objects.create(
            username=validated_data['username'],
            email=validated_data['email'],
            first_name=validated_data['first_name'],
            last_name=validated_data['last_name']
        )
        user.set_password(validated_data['password'])
        user.save()
        return user

class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = ('id', 'username', 'email', 'first_name', 'last_name', 'phone_number')

class CustomTokenObtainPairSerializer(TokenObtainPairSerializer):
    @classmethod
    def get_token(cls, user):
        token = super().get_token(user)
        # Add custom claims
        token['email'] = user.email
        return token
        
```

### Views

```python
# base/views.py
from rest_framework import generics, permissions
from rest_framework.response import Response
from rest_framework.views import APIView
from rest_framework_simplejwt.views import TokenObtainPairView
from django.contrib.auth import get_user_model
from .serializers import UserRegistrationSerializer, UserSerializer, CustomTokenObtainPairSerializer

User = get_user_model()

class UserRegistrationView(generics.CreateAPIView):
    queryset = User.objects.all()
    serializer_class = UserRegistrationSerializer
    permission_classes = [permissions.AllowAny]

class CustomTokenObtainPairView(TokenObtainPairView):
    serializer_class = CustomTokenObtainPairSerializer

class UserProfileView(APIView):
    permission_classes = [permissions.IsAuthenticated]

    def get(self, request):
        user = request.user
        serializer = UserSerializer(user)
        return Response(serializer.data)


```

### URLs

```python
# base/urls.py
from django.urls import path
from rest_framework_simplejwt.views import TokenRefreshView
from .views import UserRegistrationView, CustomTokenObtainPairView, UserProfileView

urlpatterns = [
    path('register/', UserRegistrationView.as_view(), name='register'),
    path('login/', CustomTokenObtainPairView.as_view(), name='token_obtain_pair'),
    path('token/refresh/', TokenRefreshView.as_view(), name='token_refresh'),
    path('profile/', UserProfileView.as_view(), name='profile'),
]

```

> Include these URLs in the project’s main urls.py:

```python
# myproject/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/auth/', include('base.urls')),
    # Other URLs...
]

```

## API Endpoints

```http
POST /api/auth/register/: Register a new user.
POST /api/auth/login/: Login and obtain a JWT token.
POST /api/auth/token/refresh/: Refresh the JWT token.
GET /api/auth/profile/: Get the current user's profile.
```

## Example Requests

```sh
curl -X POST http://127.0.0.1:8000/api/auth/register/ -d '{"username": "testuser", "password": "testpass123", "email": "testuser@example.com"}' -H "Content-Type: application/json"

```

```sh
curl -X POST http://127.0.0.1:8000/api/auth/login/ -d '{"username": "testuser", "password": "testpass123"}' -H "Content-Type: application/json"

```

```sh
curl -X POST http://127.0.0.1:8000/api/auth/token/refresh/ -d '{"refresh": "your_refresh_token"}' -H "Content-Type: application/json"

```

```sh
curl -H "Authorization: Bearer your_access_token" http://127.0.0.1:8000/api/auth/profile/

```